In [41]:
import LammpsPostProcess as lp
import sys
import numpy as np
import pandas as pd


def GetAtoms( filee ):
    lmpData = lp.ReadDumpFile( filee )
    lmpData.GetCords( ncount = sys.maxsize, 
                     columns = {'c_peratom[1]':'sxx','c_peratom[2]':'syy','c_peratom[3]':'szz',
                                                       'c_peratom[4]':'sxy','c_peratom[5]':'sxz','c_peratom[6]':'syz'}
                    )
    itimee=np.sum(list(lmpData.coord_atoms_broken.keys()))

    #--- volume
    rad1=0.0#AtomicRadius[1]
    rad2=0.0#AtomicRadius[2]
    rad3=0.0#AtomicRadius[3]
    !ovitos OvitosCna.py $filee Voronoi.xyz 1 3 $rad1 $rad2 $rad3  
#--- read from d2min.xyz
    ovtData = lp.ReadDumpFile( 'Voronoi.xyz' )
    ovtData.GetCords( ncount = sys.maxsize)
    #--- atom obj
    box0 = lp.Box( BoxBounds = lmpData.BoxBounds[itimee], AddMissing = np.array([0.0,0.0,0.0] ) ) #--- reference state
    atoms0 = lp.Atoms( **lmpData.coord_atoms_broken[itimee].to_dict(orient='series'),
                        AtomicVolume = ovtData.coord_atoms_broken[itimee]['AtomicVolume'].tolist())

    return atoms0, box0


path = '/Users/Home/Desktop/Tmp/txt'

atoms0, box0 = GetAtoms( '%s/dump_init.xyz'%path )

atomd = {}
boxd = {}
for dir, mode in zip(range(1,7),['xx','yy','zz','yz','xz','xy']):
    atomd[mode], boxd[mode] = GetAtoms( '%s/dump_def%s.xyz'%(path,dir) )


InputFile= /Users/Home/Desktop/Tmp/txt/dump_init.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def1.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def2.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def3.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def4.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def5.xyz
num_frames= 1
InputFile= /Users/Home/Desktop/Tmp/txt/dump_def6.xyz
num_frames= 1


In [42]:
ebulk = {}
for dir, mode in zip(range(1,7),['xx','yy','zz','yz','xz','xy']):
    if dir == 1 or dir == 2 or dir == 3:
        indxi = dir - 1
        indxj = dir - 1
    elif dir == 6: #--- xy
        indxi = 0
        indxj = 1
#    boxd[dir].__dict__
    dx=boxd[mode].CellVector[indxi,indxj]-box0.CellVector[indxi,indxj]
    l0=box0.CellVector[indxj,indxj]
        
    ebulk[mode] = dx/l0    
ebulk['yz']=ebulk['xz']=ebulk['xx']
ebulk

{'xx': -1.0000000000292e-06,
 'yy': -1.0000000000292e-06,
 'zz': -1.0000000000292e-06,
 'yz': -1.0000000000292e-06,
 'xz': -1.0000000000292e-06,
 'xy': -1.0000000000000014e-06}

In [56]:
pd.DataFrame(atomm.__dict__).head()

,x,y,z,id,type,sxx,syy,szz,sxy,sxz,syz,AtomicVolume,C66
0,0.258138,0.242737,0.251894,1.0,1.0,-7.769299e+05,-969942.534622,-1.098112e+06,22041.865025,-31496.315019,104012.472208,11.293896,107.338991
1,2.037498,2.023144,0.267614,2.0,2.0,-4.802053e+05,-443409.412469,-2.919157e+05,-58062.679538,-47039.216978,-545.914921,11.097994,83.783392
2,2.031325,0.253223,2.019011,3.0,2.0,5.008137e+05,827812.699455,9.608523e+04,-98459.162158,-204027.283398,-184863.854717,11.214992,98.460430
3,0.208169,2.010810,2.020765,4.0,3.0,1.166600e+06,610000.885685,1.495074e+06,284494.072611,-231480.080211,100802.606276,11.212058,94.772069
4,3.824685,0.228028,0.294745,5.0,2.0,-2.681080e+05,36221.936545,-5.555507e+05,-153669.544203,65723.695625,89024.259670,11.122129,101.206838


In [45]:
cfac = 1.0e-4
dxx={}
dyy={}
dzz={}
dyz={}
dxz={}
dxy={}
for mode in ['xx','yy','zz','yz','xz','xy']:
    dxx[mode]=(atomd[mode].sxx/atomd[mode].AtomicVolume-atoms0.sxx/atoms0.AtomicVolume)/ebulk[mode]*cfac
    dyy[mode]=(atomd[mode].syy/atomd[mode].AtomicVolume-atoms0.syy/atoms0.AtomicVolume)/ebulk[mode]*cfac
    dzz[mode]=(atomd[mode].szz/atomd[mode].AtomicVolume-atoms0.szz/atoms0.AtomicVolume)/ebulk[mode]*cfac
    dyz[mode]=(atomd[mode].syz/atomd[mode].AtomicVolume-atoms0.syz/atoms0.AtomicVolume)/ebulk[mode]*cfac
    dxz[mode]=(atomd[mode].sxz/atomd[mode].AtomicVolume-atoms0.sxz/atoms0.AtomicVolume)/ebulk[mode]*cfac
    dxy[mode]=(atomd[mode].sxy/atomd[mode].AtomicVolume-atoms0.sxy/atoms0.AtomicVolume)/ebulk[mode]*cfac

In [48]:
np.c_[atoms0].shape

(1, 1)

In [58]:
df=pd.DataFrame(atoms0.__dict__)
cols = list(df.keys())+['C66']
df = pd.DataFrame(np.c_[df,dxy['xy']],columns=cols)
atomm = lp.Atoms(**df)
wd = lp.WriteDumpFile(atomm, box0)
wd.Write('dump.xyz',attrs=cols)